In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\p09276\Post_doc_Yen_Fred\Projet_Machine_Learning_Julien\Portfiolo\Agentic_AI_2_0\Langchain\my_virtual_lang\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
embeddings.embed_query("hello AI")[:3]

[-0.033388201147317886, 0.034539803862571716, 0.059474579989910126]

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [12]:
my_query="Narendra modi is prime minister of india?"

In [9]:
document_embedding=embeddings.embed_documents(documents)

In [10]:
document_embedding

[[0.11998698115348816,
  -0.021302608773112297,
  -0.04288088157773018,
  0.06645584106445312,
  -0.0643523558974266,
  -0.04424867779016495,
  0.022408470511436462,
  -0.049873046576976776,
  -0.023437676951289177,
  -0.03397207707166672,
  -0.014048065058887005,
  -0.06065930798649788,
  -0.003906784579157829,
  -0.017782047390937805,
  -0.047971028834581375,
  -0.06668160855770111,
  0.004103216342628002,
  -0.013092760927975178,
  0.04439772665500641,
  0.022350657731294632,
  0.009459580294787884,
  -0.02056453377008438,
  -0.0003356038359925151,
  -0.005685813259333372,
  0.05558698996901512,
  0.02512320503592491,
  -0.002817113883793354,
  0.00875900313258171,
  0.003255280666053295,
  -0.015963423997163773,
  0.014263689517974854,
  -0.11220851540565491,
  0.08968556672334671,
  -0.031083721667528152,
  -0.02422383800148964,
  0.006152089219540358,
  0.08058717846870422,
  0.01824996992945671,
  0.05568312108516693,
  0.01670268550515175,
  0.01589597947895527,
  0.00034103207

In [13]:
query_embedding=embeddings.embed_query(my_query)

In [14]:
len(query_embedding)

384

In [15]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756665, 0.3432456 , 0.8141324 ]])

In [16]:
from sklearn.metrics.pairwise import euclidean_distances

In [17]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848288, 1.14608416, 0.60970094]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [18]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [ ]:
index=faiss.IndexFlatL2(384)


In [20]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000029FFA274150> >

In [21]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [25]:
vector_store.docstore

In [22]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['c204ded3-9885-407c-985a-4b9a355c0119',
 'dab92981-cfed-4d8f-8047-498cc462c466',
 'ab025bed-6d10-4575-82d9-e9e84490e5de']

In [23]:
vector_store.index_to_docstore_id

{0: 'c204ded3-9885-407c-985a-4b9a355c0119',
 1: 'dab92981-cfed-4d8f-8047-498cc462c466',
 2: 'ab025bed-6d10-4575-82d9-e9e84490e5de'}

In [26]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [27]:
results


[Document(id='dab92981-cfed-4d8f-8047-498cc462c466', metadata={}, page_content='AI is powerful'),
 Document(id='c204ded3-9885-407c-985a-4b9a355c0119', metadata={}, page_content='AI is future'),
 Document(id='ab025bed-6d10-4575-82d9-e9e84490e5de', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [28]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [29]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [30]:
vector_store.add_documents(documents=documents)

['9db56619-8f29-44b5-b430-d760a0cecee2',
 '3946592a-db18-48ce-b036-1441b9350cff',
 'dfeadd9a-fd06-4c7c-851a-ee05ab3fafd5',
 'cdc118d1-81de-4b5a-b9ab-5c92f3352134',
 'e77acbc8-720b-483c-a265-c8d75c0ae246',
 '01432631-f151-4d63-9181-25d3b807187e',
 '89d66205-3651-45b0-b9e9-4701707fe877',
 'c4b0f56e-9374-4caa-933c-eef157271b8a',
 '2c90f6af-90e8-46fa-89c2-85db6b377c55',
 '558bc708-6651-4ff5-a4fd-0252ad56f2d9']

In [31]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='dfeadd9a-fd06-4c7c-851a-ee05ab3fafd5', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='c4b0f56e-9374-4caa-933c-eef157271b8a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [33]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"website"} #{"$eq": "tweet"}
    
)

[Document(id='89d66205-3651-45b0-b9e9-4701707fe877', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(id='01432631-f151-4d63-9181-25d3b807187e', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.')]

In [34]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [35]:
result[0].metadata

{'source': 'news'}

In [36]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [38]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [39]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='dfeadd9a-fd06-4c7c-851a-ee05ab3fafd5', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='c4b0f56e-9374-4caa-933c-eef157271b8a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='558bc708-6651-4ff5-a4fd-0252ad56f2d9', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

inmemory(server)
ondisk(server)
cloud(yet to discuss)

In [40]:
vector_store.save_local("My_VectorStore")

In [43]:
new_vector_store=FAISS.load_local(
  "My_VectorStore",embeddings ,allow_dangerous_deserialization=True
)

In [44]:
new_vector_store.similarity_search("langchain")

[Document(id='dfeadd9a-fd06-4c7c-851a-ee05ab3fafd5', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='c4b0f56e-9374-4caa-933c-eef157271b8a', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='e77acbc8-720b-483c-a265-c8d75c0ae246', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='89d66205-3651-45b0-b9e9-4701707fe877', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [45]:
from langchain_community.document_loaders import PyPDFLoader

In [46]:
FILE_PATH=r"C:\Users\p09276\Post_doc_Yen_Fred\Projet_Machine_Learning_Julien\Portfiolo\Agentic_AI_2_0\Langchain\llama2.pdf"

In [47]:
loader=PyPDFLoader(FILE_PATH)

In [48]:
len(loader.load())

77

In [49]:
pages=loader.load()

In [50]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [51]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [52]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [53]:
split_docs = splitter.split_documents(pages)

In [54]:
len(split_docs)

615

In [55]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [56]:
vector_store.add_documents(documents=split_docs)

['7c65b602-e028-4f61-8acd-fdbb8d004d1f',
 '474b44c2-323b-4a96-a6ae-b24af4c92833',
 '7558c8f7-f17a-4a82-ba59-e8b93e6ee166',
 'a1f8cc9d-2cc8-4eac-851f-b3dcafc6e291',
 'b8b6d3e0-97a2-4f39-9128-d649f892e4be',
 '1047ff36-07d5-4ab3-b5c5-aba365c246f4',
 'f640b17a-7af9-446d-a6a6-ac90f5b9a10e',
 '82c3c042-099d-4e52-835f-5c5ebefa3240',
 '29ebea52-5d4d-485b-9b29-299a76c6a1f3',
 '15a40866-30e8-4d06-bec1-6f19934d994b',
 'cfdae17e-8ab1-4785-98ae-0fbec8ca4609',
 '861b7fba-40b3-4628-9d00-7b50aea364d2',
 '8f3f5a55-c100-43a3-a080-1cf83247a9bd',
 '25e73a48-2bbb-4329-8ad9-0028876f8ed7',
 '21b9ce24-e1b3-406d-be16-e745eb08873c',
 '8ad72b19-c51c-4dea-8077-eb4e99b3b2f8',
 '100e733f-d263-4ecc-8bed-a9a8df73436c',
 '4ab0e5fa-ed11-4437-a3ec-c61c7dfa7531',
 '0609b9f5-a032-4608-8f1c-e5160d0537ba',
 '7bb058c9-aa1c-4130-80c0-b798f3699256',
 '59e20d59-1f87-40d3-8b87-73d8fdfc7850',
 'b5697c71-a7f8-469b-a6b4-b66017401cf0',
 '7c23d882-146b-4b0f-891e-a58ef1e61c5f',
 '7adba29f-d535-4f2f-b6d1-c995437d2cff',
 '10f5f686-a43f-

In [57]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [58]:
retriever.invoke("what is llama model?")

[Document(id='10f5f686-a43f-446a-bdab-5d1a54a027dd', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\p09276\\Post_doc_Yen_Fred\\Projet_Machine_Learning_Julien\\Portfiolo\\Agentic_AI_2_0\\Langchain\\llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='c9e24d56-01ea-484a-8e5d-25a4fcecc671', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref

In [78]:
load_dotenv()
os.environ["Google_API_KEY"] = os.getenv("Google_API_KEY")
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
model

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002A002D08260>, default_metadata=(), model_kwargs={})

In [71]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [72]:
import pprint


In [73]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [74]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

context(retriever),prompt(hub),model(google),parser(langchain)

In [75]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [79]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [80]:
rag_chain.invoke("what is llama model?")

'Llama is a large language model developed by Meta.  It has various sizes, from 7B to 70B parameters, and comes in both pretrained and tuned versions.  Tuned versions, like Llama 2-Chat, are optimized for helpfulness and safety in chat applications.'